In [ ]:
# https://yhkim4504.tistory.com/5

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary

In [84]:
x = torch.randn(8,3,224,224)
x.shape

torch.Size([8, 3, 224, 224])

In [85]:
patch_size = 16 # 16 pixels

print('x :', x.shape)
patches = rearrange(x, 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=patch_size, s2=patch_size)
print('patches :', patches.shape)

x : torch.Size([8, 3, 224, 224])
patches : torch.Size([8, 196, 768])


In [33]:
patch_size = 16
in_channels = 3
emb_size = 768
conv1 = nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size)
x1 = conv1(x)
print(x1.shape)

x2 = x1.permute(0,2,3,1).view(8, 14*14, 768)
print(x2.shape)

x3 = Rearrange('b e (h) (w) -> b (h w) e')(x1)
print(x2.shape)

torch.Size([8, 768, 14, 14])
torch.Size([8, 196, 768])
torch.Size([8, 196, 768])


In [34]:
x4 = rearrange(x1, 'b c (h s1) (w s2) -> b (h w) (s1 s2 c)', s1=1, s2=1)
x4.shape

torch.Size([8, 196, 768])

In [35]:
(x2 == x4).sum()

tensor(1204224)

In [36]:
(x2 == x3).sum()

tensor(1204224)

In [19]:
patch_size = 16
in_channels = 3
emb_size = 768

projection = nn.Sequential(
            # using a conv layer instead of a linear one -> performance gains
            nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e (h) (w) -> b (h w) e'),
        )

y = projection(x)
y.shape

torch.Size([8, 196, 768])

In [24]:
# print(x2[0][0])


In [25]:
# print(y[0][0])

In [37]:
emb_size = 768
img_size = 224
patch_size = 16


In [80]:
cls_token = nn.Parameter(torch.randn(1,1,emb_size))
cls_token.shape
position = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))
# position = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))
position.shape

cls_tokens = cls_token.repeat(8,1,1)

tot_tokens = torch.cat([cls_tokens, x2], dim=1)
tot_tokens.shape

tot_tokens += position

In [78]:
cls_tokens[0][0][0]
cls_tokens[1][0][0]
cls_tokens[2][0][0]
cls_tokens[0:8,0,0]

tensor([-0.8504, -0.8504, -0.8504, -0.8504, -0.8504, -0.8504, -0.8504, -0.8504],
       grad_fn=<SelectBackward>)

In [86]:
# 이미지를 패치사이즈로 나누고 flatten
print(x.shape)
projected_x = projection(x)
print('Projected X shape :', projected_x.shape)

# cls_token과 pos encoding Parameter 정의
cls_token = nn.Parameter(torch.randn(1,1, emb_size))
positions = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))
print('Cls Shape :', cls_token.shape, ', Pos Shape :', positions.shape)

# cls_token을 반복하여 배치사이즈의 크기와 맞춰줌
batch_size = 8
cls_tokens = repeat(cls_token, '() n e -> b n e', b=batch_size)
print('Repeated Cls shape :', cls_tokens.shape)

# cls_token과 projected_x를 concatenate
cat_x = torch.cat([cls_tokens, projected_x], dim=1)

# position encoding을 더해줌
cat_x += positions
print('output : ', cat_x.shape)

torch.Size([8, 3, 224, 224])
Projected X shape : torch.Size([8, 196, 768])
Cls Shape : torch.Size([1, 1, 768]) , Pos Shape : torch.Size([197, 768])
Repeated Cls shape : torch.Size([8, 1, 768])
output :  torch.Size([8, 197, 768])


In [87]:
emb_size = 768
num_heads = 8

keys = nn.Linear(emb_size, emb_size)
queries = nn.Linear(emb_size, emb_size)
values = nn.Linear(emb_size, emb_size)
print(keys, queries, values)

Linear(in_features=768, out_features=768, bias=True) Linear(in_features=768, out_features=768, bias=True) Linear(in_features=768, out_features=768, bias=True)


In [96]:
print(x2.shape)
queries(cat_x).shape

torch.Size([8, 196, 768])


TypeError: 'Tensor' object is not callable

In [91]:
queries = rearrange(queries(x2), "b n (h d) -> b h n d", h=num_heads)
keys = rearrange(keys(x2), "b n (h d) -> b h n d", h=num_heads)
values  = rearrange(values(x2), "b n (h d) -> b h n d", h=num_heads)

print('shape :', queries.shape, keys.shape, values.shape)

TypeError: 'Tensor' object is not callable

In [2]:
import torch
a = torch.randn(4, 4)
a
a = torch.tensor([[ 1.3398,  0.2663, -0.2686,  0.2450],
        [-0.7401, -0.8805, -0.3402, -1.1936],
        [ 0.4907, -1.3948, -1.0691, -0.3132],
        [-1.6092,  0.5419, -0.2993,  0.3195]])


In [4]:
torch.argmax(a, dim=-1)

tensor([0, 2, 0, 1])

In [6]:
a = torch.randn(2,4,3)
torch.argmax(a, dim=-1)

tensor([[2, 0, 1, 2],
        [0, 1, 1, 0]])